# 1. 패키지 설치

In [1]:
%pip install python-docx python-dotenv tiktoken chromadb openai


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- LangChain의 TextSplitter를 사용할 수 없기 때문에 [python-docx](https://github.com/python-openxml/python-docx)와 [tiktoken](https://github.com/openai/tiktoken)을 활용해서 chunk 생성

In [2]:
from docx import Document

document = Document('./tax.docx')

full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    full_text += f'{paragraph.text}\n'
    

In [3]:
import tiktoken 

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    
    return text_list

In [4]:
chunk_list = split_text(full_text, 1500)

In [5]:
import chromadb

chroma_client = chromadb.Client()

In [6]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [7]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [8]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [9]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')


In [12]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`query`)를 활용해서 가져옴
    - method이름은 `query`지만 유사도 검색 실행

In [13]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [14]:
retrieved_doc['documents'][0]

[' 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품\n나. 「국가보안법」에 따라 받는 상금과 보로금\n다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상\n라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.\n마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품\n바. 「문화재보호법」에 따라 국가지정문화재로 지정된 서화ㆍ골동품의 양도로 발생하는 소득\n사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득\n아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득\n\u3000\u3000\u3000\u30001) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금\n\u3000\u3000\u3000\u30002) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대\n\u3000\u3000\u3000\u30003) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액\n\u3000\u3000\u3000\u30004) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액\n\u3000\u3000\u3000\u30005) 종교관련종사자가 기획재정부령으로 정하는 사택을 제공받아 얻

# 4. Augmentation을 위한 Prompt 활용 및 답변 생성

- Retrieval된 데이터는 system prompt에 추가해서 LLM의 배경지식으로 활용
- Large Language Model은 사용자 질문에 대해 system prompt에 있는 배경지식을 활용하여 답변 생성

In [15]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
    {"role": "user", "content": query}
  ]
)

In [16]:
response.choices[0].message.content

'연봉 5천만 원인 직장인의 소득세를 계산하려면 다음 단계를 따라야 합니다:\n\n1. **총 급여액에서 근로소득공제 계산**:\n   \\[\n   \\text{연봉 5천만 원에 대한 근로소득공제는 다음과 같습니다.}\n   \\]\n   \\[\n   \\text{총급여(5천만 원)에 대한 근로소득공제는 } \\\\\n   3,300만원 이하 부분은 55% 공제: 3,300만원 × 55\\% = 1,815만원 \\\\\n   3,300만원 초과 7,000만원 이하 부분은 30% 공제: (5000만원 - 3300만원) × 30\\% = 510만원 \\\\\n   따라서,\n   \\]\n   \\[\n   \\text{근로소득공제액} = 1,815만원 + 510만원 = 2,325만원\n   \\]\n\n2. **과세표준 계산**:\n   \\[\n   \\text{과세표준} = \\text{연봉} - \\text{근로소득공제액}\n   \\]\n   \\[\n   5000만원 - 2,325만원 = 2,675만원\n   \\]\n\n3. **기본 세율 적용 및 산출세액 계산**:\n\n   \\[\n   \\text{구간별 세율}\n   \\]\n   - 1,200만원 이하는 6%\n   - 1,200만원 초과 ~ 4,600만원 이하는 15%\n   - 4,600만원 초과 ~ 8,800만원 이하는 24%\n   - (이하 생략 - 상위 구간에 해당되지 않음)\n\n   \\[\n   \\text{과세표준 2,675만원이므로,}\n   \\]\n   \\[\n   1,200만원 이하는 6%: 1,200만원 × 0.06 = 72만원\n   \\]\n   \\[\n   1,200만원 초과 ~ 2,675만원 이하는 15%: (2,675만원 - 1,200만원) × 0.15 = 221.25만원\n   \\]\n   \\[\n   \\text{따라서, 산출세액} = 72만원 + 221.25만원 = 293.25만원\n   \\]\n\n4. **세액 공제 적용**:\n   \